In [14]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties":{
                    "company_name": {
                        "type":"string",
                        "description":"The name of the company"
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

# get_income_statement({"ticker":"AAPL"})

In [6]:
from openai import OpenAI

client = OpenAI()

# assistant = client.beta.assistants.create(
# name="Investor Assistant",
# instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
# tools=functions,
# model="gpt-4o",
# )

assistant_id = "asst_13eo2MdAQaNGc4XTBhTHlOW3"



In [7]:
client = OpenAI()

thread = client.beta.threads.create(
    messages = [
        {
            "role":"user",
            "content":"I want to know if the Salesforce stock is a good buy"
        }
    ]
)

thread

Thread(id='thread_6Gacw0IutbyrLlbxArBw3MK0', created_at=1727835056, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None), _request_id='req_30e9a8ce64ca65b638260c4c96f76d99', __exclude_fields__={'__exclude_fields__', '_request_id'})

In [16]:

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)
run

Run(id='run_Pk9EsB1tKOTx3rNHnNIo8EVk', assistant_id='asst_13eo2MdAQaNGc4XTBhTHlOW3', cancelled_at=None, completed_at=None, created_at=1727835934, expires_at=1727836534, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_6Gacw0IutbyrLlbxArBw3MK0', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(

In [23]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id
    )

def send_messages(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        #print(action)
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append({
            "output":functions_map[function.name](json.loads(function.arguments)),
            "tool_call_id":action_id
        })
    return outputs

def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs
    )

In [27]:
get_run(run.id, thread.id).status

'completed'

In [28]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: To determine if Salesforce (CRM) is a good buy, let's analyze the provided financial data:

### Income Statement Analysis
#### Key Figures for Recent Years:
* **2023 (latest data)**
  - Total Revenue: \$34.857 billion
  - Net Income: \$4.136 billion
  - EBITDA: \$9.958 billion
  - Diluted EPS: \$4.20

* **2022**
  - Total Revenue: \$31.352 billion
  - Net Income: \$208 million
  - EBITDA: \$5.644 billion
  - Diluted EPS: \$0.21

* **2021**
  - Total Revenue: \$26.492 billion
  - Net Income: \$1.444 billion
  - EBITDA: \$3.846 billion
  - Diluted EPS: \$1.48

**Trend Analysis:**
- Revenue has shown consistent growth over the past few years.
- Significant increase in net income from 2022 to 2023.
- EBITDA and EPS both show significant improvement, indicating better operational efficiency and profitability.

### Balance Sheet Analysis
#### Key Figures for Recent Years:
* **2023 (latest data)**
  - Total Assets: \$99.823

In [25]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


Run(id='run_Pk9EsB1tKOTx3rNHnNIo8EVk', assistant_id='asst_13eo2MdAQaNGc4XTBhTHlOW3', cancelled_at=None, completed_at=None, created_at=1727835934, expires_at=1727836534, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1727835934, status='queued', thread_id='thread_6Gacw0IutbyrLlbxArBw3MK0', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefin